# Assignment 7: CNNs & RNNs

In [28]:
# enter your name and UFL email address
name = 'enter your name'
email = 'enter your email'

name = 'solution' # ###- 
email = 'solution' # ###- 

In [29]:
if name == 'enter your name' or email == 'enter your email':
    assert False, 'Enter your name & email first!'
else:
    print('Assignment 7 -- name: {}, email: {}\n'.format(name, email))
    
    # Load packages we need
    import sys
    import os
    import time

    import numpy as np
    import sklearn
    
    # we'll use tensorflow and keras for neural networks
    import tensorflow as tf
    import tensorflow.keras as keras
    
    # import layers we may use
    from tensorflow.keras.layers import Input, Flatten, Dense, Conv2D, MaxPooling2D, Dropout

    # import callbacks we may use
    from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
    
    # Load the TensorBoard notebook extension
    #%load_ext tensorboard

    from matplotlib import pyplot as plt
    plt.rcParams.update({'font.size': 16})

    # Let's check our software versions
    print('### Python version: ' + __import__('sys').version)
    print('### NumPy version: ' + np.__version__)
    print('### Scikit-learn version: ' + sklearn.__version__)
    print('### Tensorflow version: ' + tf.__version__)
    print('### TF Keras version: ' + keras.__version__)
    print('------------')


    # load our packages / code
    sys.path.insert(1, '../common/')
    import utils
    import plots

Assignment 7 -- name: solution, email: solution

### Python version: 3.8.5 (default, Jan 27 2021, 15:41:15) 
[GCC 9.3.0]
### NumPy version: 1.19.5
### Scikit-learn version: 0.24.0
### Tensorflow version: 2.4.0
### TF Keras version: 2.4.0
------------


In [30]:
# global parameters to control behavior of the pre-processing, ML, analysis, etc.
seed = 42

# deterministic seed for reproducibility
np.random.seed(seed)
tf.random.set_seed(seed)

prop_vec = [24, 2, 2]

## [Task 1] (20 points) Loading and Processing CIFAR-10

### [Task 1a] (20 points) Complete the implementation of load_preprocess_cifar10(). Make sure you correctly implement all of the cases.

In [31]:
from tensorflow.keras.datasets import cifar10

# refer to: https://www.tensorflow.org/api_docs/python/tf/keras/datasets/cifar10/load_data
# and to https://www.cs.toronto.edu/~kriz/cifar.html
def load_preprocess_cifar10(onehot=True, minmax_normalize=True):
    
    ###* put your code here (~10-20 lines) *###
    labels = np.array(['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'])
    train, testval = cifar10.load_data()
    
    train_x, train_y = train
    testval_x, testval_y = testval
    
    if minmax_normalize:
        train_x = train_x / 255.0
        testval_x = testval_x / 255.0
        
    if onehot:
        train_y = keras.utils.to_categorical(train_y, labels.shape[0])
        testval_y = keras.utils.to_categorical(testval_y, labels.shape[0])
    
    # split test - val
    nval = testval_x.shape[0] // 2
    
    val_x = testval_x[:nval]
    val_y = testval_y[:nval]
    
    test_x = testval_x[nval:]
    test_y = testval_y[nval:]
    
    return train_x, train_y, test_x, test_y, val_x, val_y, labels

In [32]:
# do some sanity checks
train_x, train_y, test_x, test_y, val_x, val_y, labels = load_preprocess_cifar10(onehot=False, minmax_normalize=False)
assert train_x.shape[0] == train_y.shape[0] and test_x.shape[0] == test_y.shape[0] and val_x.shape[0] == val_y.shape[0]
assert np.amax(train_x) >= 255 and np.amax(test_x) >= 255 and np.amax(val_x) >= 255
assert train_y.shape == (train_y.shape[0],) or train_y.shape == (train_y.shape[0],1)

train_x, train_y, test_x, test_y, val_x, val_y, labels = load_preprocess_cifar10(onehot=True, minmax_normalize=False)
assert np.amax(train_x) >= 255 and np.amax(test_x) >= 255 and np.amax(val_x) >= 255
assert train_y.shape == (train_y.shape[0],10) and train_y.shape[1] == test_y.shape[1]

In [33]:
# actually load the data
train_x, train_y, test_x, test_y, val_x, val_y, labels = load_preprocess_cifar10()
assert np.amax(train_x) <= 1 and np.amax(test_x) <= 1 and np.amax(val_x) <= 1
assert np.amax(train_x) >= 0 and np.amax(test_x) >= 0 and np.amax(val_x) >= 0

assert labels.shape[0] == 10 and labels.shape[0] == train_y.shape[1]

In [34]:
print(train_x.shape, val_x.shape, test_x.shape, train_y.shape)

(50000, 32, 32, 3) (5000, 32, 32, 3) (5000, 32, 32, 3) (50000, 10)


## [Task 2] (25 points) Training a CNN for Cifar-10

#### We will use the following architecture
- Conv layer with 32 filters, (3,3) filter size, stride of 1, padding 'same'
- Conv layer with 32 filters, (3,3) filter size, stride of 1, padding 'same'
- Max pooling layer (2,2)
- Dropout with rate 25%
- Conv layer with 64 filters, (3,3) filter size, stride of 1, padding 'same'
- Conv layer with 64 filters, (3,3) filter size, stride of 1, padding 'same'
- Max pooling layer (2,2)
- Dropout with rate 25%
- Conv layer with 128 filters, (3,3) filter size, stride of 1, padding 'same'
- Conv layer with 128 filters, (3,3) filter size, stride of 1, padding 'same'
- Max pooling layer (2,2)
- Dropout with rate 25%
- Flatten
- FC with 128 units
- Dropout with rate 25%
- FC with 64 units
- Dropout with rate 25%
- (Output layer) FC with 10 units

#### For all layers (if applicable) except the output layer you should use:
- ReLU as activation function
- He uniform weight initialization strategy
- L2 regularization with regularization constant set to 0.001

#### For the output layer you should select a suitable activation function that is consistent with the task and loss function you use. Use Adam for the optimizer with learning rate 0.002.

### [Task 2a] (15 points) Implement create_compile_cnn()

In [35]:
def create_compile_cnn(input_shape=[32, 32, 3], num_outputs=10, verbose=False):
    
    model = keras.models.Sequential(name='CIFAR-10--CNN')
    
    ### Don't forget to compile the model and print the summary if verbose=True
    ###* put your code here (~20 lines) *###
    
    l2reg = keras.regularizers.l2(0.001)
    
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l2reg, padding='same', 
                     input_shape=input_shape))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l2reg, padding='same', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.25, name='dropout-conv1'))
    
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l2reg, padding='same'))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l2reg, padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.25, name='dropout-conv2'))
    
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l2reg, padding='same'))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l2reg, padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.25, name='dropout-conv3'))
    
    model.add(Flatten(name='flatten'))
    
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l2reg, name='fc1'))
    model.add(Dropout(0.25, name='dropout-fc1'))
    model.add(Dense(64, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l2reg, name='fc2'))
    model.add(Dropout(0.25, name='dropout-fc2'))
    
    model.add(Dense(num_outputs, activation='softmax', name='output'))
    
    opt = keras.optimizers.Adam(lr=0.002)
    
    if verbose:
        model.summary()
    
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    
    return model

In [36]:
_ = create_compile_cnn(verbose=True)

Model: "CIFAR-10--CNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
dropout-conv1 (Dropout)      (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 64)        

### [Task 2b] (10 points) Train the model. Fill in the implementation below.

In [37]:
cnn_model_fp = './cifar10-cnn.h5'

# If the model file exists, load it. Otherwise train it and save the model.
# Note: if you need to retrain the model, simply delete the h5 file.
if os.path.exists(cnn_model_fp):
    model = keras.models.load_model(cnn_model_fp)
else:
    model = create_compile_cnn(verbose=False)
    
    # do the training using model.fit() for at least 3 epochs and your chosen batch_size
    # you can set any callback you want on it, including checkpoint, early stopping, etc.
    ###* put your code here (~3-5 lines) *###
    early_stop_cb = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)

    max_epochs = 15
    batch_size = 64

    hist = model.fit(train_x, train_y, epochs=max_epochs, batch_size=batch_size, validation_data=(val_x, val_y), callbacks=[early_stop_cb])
    
    # save the model
    model.save(cnn_model_fp)

Epoch 1/15
782/782 [==============================] - 157s 199ms/step - loss: 2.7462 - accuracy: 0.2010 - val_loss: 1.7624 - val_accuracy: 0.4140
Epoch 2/15
782/782 [==============================] - 172s 220ms/step - loss: 1.7836 - accuracy: 0.3962 - val_loss: 1.5142 - val_accuracy: 0.5056
Epoch 3/15
782/782 [==============================] - 170s 217ms/step - loss: 1.6218 - accuracy: 0.4687 - val_loss: 1.4836 - val_accuracy: 0.5324
Epoch 4/15
782/782 [==============================] - 169s 216ms/step - loss: 1.5293 - accuracy: 0.5078 - val_loss: 1.3760 - val_accuracy: 0.5802
Epoch 5/15
782/782 [==============================] - 173s 221ms/step - loss: 1.4676 - accuracy: 0.5455 - val_loss: 1.3058 - val_accuracy: 0.5972
Epoch 6/15
782/782 [==============================] - 172s 220ms/step - loss: 1.4297 - accuracy: 0.5590 - val_loss: 1.2490 - val_accuracy: 0.6176
Epoch 7/15
782/782 [==============================] - 173s 222ms/step - loss: 1.3882 - accuracy: 0.5799 - val_loss: 1.2773 -

In [ ]:
# let's evaluate the model on the test data
loss, acc = model.evaluate(test_x, test_y, verbose=0)
print('[Model] Test accuracy: {:.2f}%'.format(100*acc))

## [Task 3] (15 points) Processing Sequence Data

### [Task 3a] (15 points) Fill in the implementation of load_preprocess_imdb()

In [43]:
from tensorflow.keras.datasets import imdb

# the size of the vocabulary we'll use
vocab_size = 12000
maxlen = 150

# refer to: https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb
def load_preprocess_imdb(num_words=vocab_size, prop_vec=prop_vec, maxlen=maxlen, vectorize=False):
    
    np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)
    
    train, testval = imdb.load_data(num_words=num_words, maxlen=maxlen, oov_char=0)
    
    np.warnings.filterwarnings('default', category=np.VisibleDeprecationWarning)    
    
    ### Process the data 
    ### Merge train and testval, but then split again into train, test, val sets (according to prop_vec). You can use utils.train_test_val_split().)
    ### - If vectorize=True, then you must encode the features of each example into vectors of vocab_size entries 
    ### such that entry i contains the number of time word i appeared in the sequence
    ### - If vectorize=False, then you must encode the features of each examples as a sequence of size maxlen (represented as a np.array()). 
    ### Make sure to pad sequences with 0 as appropriate.
    ###* put your code here (~10-15 lines) *###
    
    def seq_to_vec_count(x):
        ret = np.zeros((x.shape[0], num_words))
        for i, seq in enumerate(x):
            for x in seq:
                ret[i, x] += 1
        return ret
    
    def seq_to_2darray(x):
        ret = np.zeros((x.shape[0], maxlen))
        for i, seq in enumerate(x):
            ret[i,:] = np.pad(seq, (0, maxlen - len(seq)), 'constant')
        return ret    
    
    
    train_x, train_y = train
    testval_x, testval_y = testval
    
    if vectorize:
        train_x = seq_to_vec_count(train_x)
        testval_x = seq_to_vec_count(testval_x)
    else:
        train_x = seq_to_2darray(train_x)
        testval_x = seq_to_2darray(testval_x)
    
    # put things back together
    all_x = np.r_[np.array(train_x), np.array(testval_x)]
    all_y = np.r_[np.array(train_y), np.array(testval_y)]
    
    
    # split the data into train, test, val
    train_x, train_y, test_x, test_y, val_x, val_y = utils.train_test_val_split(all_x, all_y, prop_vec, shuffle=True, seed=seed)
    
    return train_x, train_y, test_x, test_y, val_x, val_y

In [44]:
# sanity checks
train_x, train_y, test_x, test_y, val_x, val_y = load_preprocess_imdb(vectorize=False)
assert train_x.shape == (16281, maxlen) and train_y.shape == (train_x.shape[0],)

train_x, train_y, test_x, test_y, val_x, val_y = load_preprocess_imdb(vectorize=True)
assert train_x.shape == (16281, vocab_size) and train_y.shape == (train_x.shape[0],)

## [Task 4] (35 points) RNN for Sentiment Analysis

In [46]:
# let's load the data
train_x, train_y, test_x, test_y, val_x, val_y = load_preprocess_imdb(vectorize=False)

### [Task 4a] (35 points) Complete the code below to define an RNN architecture for sentiment analysis. The goal is to predict the sentiment of IMDB reviews. You can use any architecture you want, but a good place to start would be to use an embedding layer followed by some recurrent layers (e.g., LSTM, GRU, etc.). Keep the number of parameters of the model below 2m.

In [47]:
def create_compile_rnn(input_shape=[None], vocab_size=vocab_size, embedding_size=128, num_outputs=1, verbose=False):
    
    model = keras.models.Sequential(name='imdb-RNN')
    
    ### Don't forget to compile the model and print the summary if verbose=True
    ###* put your code here (~20 lines) *###

    model.add(Input(shape=input_shape))
    model.add(keras.layers.Embedding(vocab_size, embedding_size, name='embedding'))
    
    model.add(keras.layers.GRU(128, return_sequences=True, name='gru1'))
    model.add(keras.layers.GRU(128, name='gru2'))
    
    model.add(Dense(num_outputs, activation='sigmoid', name='output'))
    
    opt = keras.optimizers.Adam(lr=0.002)
    
    if verbose:
        model.summary()
    
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    return model

In [48]:
model = create_compile_rnn(verbose=True)

Model: "imdb-RNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         1536000   
_________________________________________________________________
gru1 (GRU)                   (None, None, 128)         99072     
_________________________________________________________________
gru2 (GRU)                   (None, 128)               99072     
_________________________________________________________________
output (Dense)               (None, 1)                 129       
Total params: 1,734,273
Trainable params: 1,734,273
Non-trainable params: 0
_________________________________________________________________


In [49]:
early_stop_cb = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# feel free to tweak the batch size, number of epochs and callbacks.
max_epochs = 3
batch_size = 32

hist = model.fit(train_x, train_y, epochs=max_epochs, batch_size=batch_size, validation_data=(val_x, val_y), callbacks=[early_stop_cb])

Epoch 1/3
509/509 [==============================] - 133s 255ms/step - loss: 0.6936 - accuracy: 0.5084 - val_loss: 0.5022 - val_accuracy: 0.7721
Epoch 2/3
509/509 [==============================] - 137s 269ms/step - loss: 0.3485 - accuracy: 0.8539 - val_loss: 0.2516 - val_accuracy: 0.9071
Epoch 3/3
509/509 [==============================] - 139s 274ms/step - loss: 0.1227 - accuracy: 0.9597 - val_loss: 0.2656 - val_accuracy: 0.8916


In [50]:
# let's evaluate the model on the test data
loss, acc = model.evaluate(test_x, test_y, verbose=0)
print('[Model] Test accuracy: {:.2f}%'.format(100*acc))

[Model] Test accuracy: 91.30%


## [CIS6930 Additional Task -- Task 5] (25 points): DNN for Sentiment Analysis

### In the previous task, we use an RNN for sentiment analysis. In this task you will use a neural network without any recurrent layers for the same task as a comparison.

### We'll use the data in vectorized form for this.

In [51]:
# Let's load the data in vectorized form
train_x, train_y, test_x, test_y, val_x, val_y = load_preprocess_imdb(vectorize=True)

### [Task 5a] (20 points) Complete the code below to define an architecture of your choice *without* any recurrent layers. The goal is to get the best model with the fewest number of parameters. Keep the number of parameters of the model below 2m and ideally similar to the model of Task 4.

In [52]:
def create_compile_dnn(input_shape=[vocab_size], num_outputs=1, verbose=False):
    
    model = keras.models.Sequential(name='imdb-DNN')
    
    ### Don't forget to compile the model and print the summary if verbose=True
    ###* put your code here (~10 lines) *###

    model.add(Input(shape=input_shape))
    model.add(Dense(128, activation='relu', name='fc1'))
    model.add(Dropout(0.5, name='dropout1'))
    model.add(Dense(96, activation='relu', name='fc2'))
    model.add(Dropout(0.3, name='dropout2'))
    model.add(Dense(48, activation='relu', name='fc3'))
    model.add(Dropout(0.2, name='dropout3'))
    model.add(Dense(16, activation='relu', name='fc4'))
    model.add(Dropout(0.2, name='dropout4'))
    
    model.add(Dense(num_outputs, activation='sigmoid', name='output'))
    
    opt = keras.optimizers.Adam(lr=0.001)
    
    if verbose:
        model.summary()
    
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    return model

In [53]:
model = create_compile_dnn(verbose=True)

Model: "imdb-DNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
fc1 (Dense)                  (None, 128)               1536128   
_________________________________________________________________
dropout1 (Dropout)           (None, 128)               0         
_________________________________________________________________
fc2 (Dense)                  (None, 96)                12384     
_________________________________________________________________
dropout2 (Dropout)           (None, 96)                0         
_________________________________________________________________
fc3 (Dense)                  (None, 48)                4656      
_________________________________________________________________
dropout3 (Dropout)           (None, 48)                0         
_________________________________________________________________
fc4 (Dense)                  (None, 16)                784

In [54]:
early_stop_cb = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# feel free to tweak the batch size, number of epochs and callbacks.
max_epochs = 50
batch_size = 100

hist = model.fit(train_x, train_y, epochs=max_epochs, batch_size=batch_size,validation_data=(val_x, val_y), callbacks=[early_stop_cb])

Epoch 1/50
163/163 [==============================] - 3s 16ms/step - loss: 0.5746 - accuracy: 0.6777 - val_loss: 0.2824 - val_accuracy: 0.8850
Epoch 2/50
163/163 [==============================] - 2s 15ms/step - loss: 0.2312 - accuracy: 0.9183 - val_loss: 0.2706 - val_accuracy: 0.8886
Epoch 3/50
163/163 [==============================] - 2s 15ms/step - loss: 0.1568 - accuracy: 0.9473 - val_loss: 0.3100 - val_accuracy: 0.8857
Epoch 4/50
163/163 [==============================] - 2s 15ms/step - loss: 0.0999 - accuracy: 0.9687 - val_loss: 0.3537 - val_accuracy: 0.8842
Epoch 5/50
163/163 [==============================] - 2s 15ms/step - loss: 0.0760 - accuracy: 0.9750 - val_loss: 0.3679 - val_accuracy: 0.8813
Epoch 6/50
163/163 [==============================] - 2s 15ms/step - loss: 0.0644 - accuracy: 0.9803 - val_loss: 0.4491 - val_accuracy: 0.8805
Epoch 7/50
163/163 [==============================] - 2s 15ms/step - loss: 0.0490 - accuracy: 0.9839 - val_loss: 0.5000 - val_accuracy: 0.8850

In [55]:
# let's evaluate the model on the test data
loss, acc = model.evaluate(test_x, test_y, verbose=0)
print('[Model] Test accuracy: {:.2f}%'.format(100*acc))

[Model] Test accuracy: 90.79%


### [Task 5b] (5 points) Compare this model to the model of Task 4. What do you conclude?

In [56]:
###* put your answer here *###
#
# The RNN model is (slightly) better in terms of performance but takes much longer to train. 
# 
# A plausible explanation is that even though the data is sequential, the order of words do not matter 
# that much to whether a review is positive or negative, thus ignoring the sequential nature of the data 
# does not harm model performance significantly. 
#